In [1]:
#https://www.machinelearningplus.com/nlp/topic-modeling-gensim-python/

In [1]:
# Run in python console
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\dastous\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [12]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [4]:
df_review=pd.read_excel("../CannaConnect/Dataset/description_clean.xlsx")
df=df_review.Description.astype(str)
df.head()

0     og   hybrid strain pack strong punch name sup...
1     aloha white widow especially potent cut white...
2     sativa dominant hybrid bred spain medical see...
3     dawgs hybrid g chemdawg genetics bred canadia...
4    known kosher tangie k gold  indica dominant hy...
Name: Description, dtype: object

In [6]:
# Convert to list
data = df.values.tolist()

In [7]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['og', 'hybrid', 'strain', 'pack', 'strong', 'punch', 'name', 'supposedly', 'refers', 'strength', 'high', 'price', 'first', 'started', 'showing', 'hollywood', 'plant', 'og', 'tends', 'produce', 'large', 'dark', 'green', 'bud', 'stem', 'user', 'report', 'strong', 'body', 'effect', 'indica', 'pain', 'relief', 'alert', 'cerebral', 'feeling', 'thanks', 'sativa', 'side']]


In [8]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['og', 'hybrid', 'strain', 'pack', 'strong', 'punch', 'name', 'supposedly', 'refers', 'strength', 'high', 'price', 'first', 'started', 'showing', 'hollywood', 'plant', 'og', 'tends', 'produce', 'large', 'dark', 'green', 'bud', 'stem', 'user_report', 'strong', 'body', 'effect', 'indica', 'pain', 'relief', 'alert', 'cerebral', 'feeling', 'thanks', 'sativa', 'side']


In [13]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [14]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['hybrid', 'strain', 'pack', 'strong', 'punch', 'name', 'supposedly', 'refer', 'strength', 'high', 'price', 'first', 'start', 'show', 'hollywood', 'plant', 'tend', 'produce', 'large', 'dark', 'green', 'bud', 'stem', 'user_report', 'strong', 'body', 'effect', 'indica', 'pain', 'relief', 'alert', 'cerebral', 'feeling', 'thank', 'side']]


In [15]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 2), (30, 1), (31, 1), (32, 1), (33, 1)]]


In [16]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('alert', 1),
  ('body', 1),
  ('bud', 1),
  ('cerebral', 1),
  ('dark', 1),
  ('effect', 1),
  ('feeling', 1),
  ('first', 1),
  ('green', 1),
  ('high', 1),
  ('hollywood', 1),
  ('hybrid', 1),
  ('indica', 1),
  ('large', 1),
  ('name', 1),
  ('pack', 1),
  ('pain', 1),
  ('plant', 1),
  ('price', 1),
  ('produce', 1),
  ('punch', 1),
  ('refer', 1),
  ('relief', 1),
  ('show', 1),
  ('side', 1),
  ('start', 1),
  ('stem', 1),
  ('strain', 1),
  ('strength', 1),
  ('strong', 2),
  ('supposedly', 1),
  ('tend', 1),
  ('thank', 1),
  ('user_report', 1)]]

In [17]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=15, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [18]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.064*"flower" + 0.056*"bud" + 0.045*"high" + 0.042*"plant" + 0.042*"time" '
  '+ 0.037*"grow" + 0.034*"seed" + 0.034*"produce" + 0.031*"yield" + '
  '0.029*"week"'),
 (1,
  '0.128*"cup" + 0.056*"result" + 0.042*"place" + 0.030*"prize" + '
  '0.029*"functional" + 0.029*"minor" + 0.028*"world" + 0.026*"uplifting" + '
  '0.025*"big" + 0.024*"connoisseur"'),
 (2,
  '0.073*"trainwreck" + 0.062*"sensation" + 0.052*"fatigue" + 0.036*"enough" + '
  '0.036*"kick" + 0.034*"generous" + 0.033*"delicious" + 0.031*"alert" + '
  '0.030*"northern" + 0.030*"usually"'),
 (3,
  '0.158*"blue" + 0.051*"sativas" + 0.051*"bank" + 0.043*"musky" + '
  '0.028*"bloom" + 0.027*"introspective" + 0.022*"mature" + 0.017*"pua_mana" + '
  '0.014*"uplifted" + 0.010*"positive"'),
 (4,
  '0.126*"thai" + 0.090*"short" + 0.055*"outdoor" + 0.045*"together" + '
  '0.034*"harvest" + 0.031*"october" + 0.028*"soil" + 0.026*"fully" + '
  '0.016*"hydroponic" + 0.014*"award"'),
 (5,
  '0.102*"expect" + 0.086*"new" + 0.073

In [19]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -10.531111034308498

Coherence Score:  0.40042959324855404


In [20]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\dastous\Anaconda3\envs\py3.6\lib\site-packages\pyLDAvis\_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.377574  0.293476       1        1  59.825577
0     -0.293679 -0.076577       2        1  11.264098
14    -0.155341 -0.251061       3        1   7.246403
12    -0.107866 -0.172663       4        1   6.355774
7     -0.010199 -0.038379       5        1   3.296574
6      0.010231 -0.045136       6        1   3.275892
1      0.032791  0.028923       7        1   2.264673
11     0.090431  0.027194       8        1   1.395702
2      0.094828  0.030214       9        1   1.236204
10     0.094146  0.029843      10        1   1.150750
5      0.116934  0.034480      11        1   0.816093
3      0.123610  0.035161      12        1   0.737939
4      0.121779  0.034007      13        1   0.589361
9      0.128264  0.035305      14        1   0.311293
13     0.131645  0.035212      15        1   0.233669, topic_info=     Category         Freq           Term        Total  loglift  logprob
570   Default   776.000000          white   776.000000  30.0000  30.0000
700   Default   388.000000           haze   388.000000  29.0000  29.0000
27    Default  3829.000000         strain  3829.000000  28.0000  28.0000
2     Default   855.000000            bud   855.000000  27.0000  27.0000
139   Default   704.000000         flower   704.000000  26.0000  26.0000
9     Default   842.000000           high   842.000000  25.0000  25.0000
74    Default   441.000000          skunk   441.000000  24.0000  24.0000
236   Default   602.000000       cannabis   602.000000  23.0000  23.0000
73    Default   863.000000           seed   863.000000  22.0000  22.0000
473   Default   286.000000            cup   286.000000  21.0000  21.0000
17    Default   467.000000          plant   467.000000  20.0000  20.0000
370   Default   460.000000           time   460.000000  19.0000  19.0000
5     Default  2088.000000         effect  2088.000000  18.0000  18.0000
50    Default   410.000000           grow   410.000000  17.0000  17.0000
237   Default   318.000000            cbd   318.000000  16.0000  16.0000
110   Default   316.000000            thc   316.000000  15.0000  15.0000
19    Default   384.000000        produce   384.000000  14.0000  14.0000
11    Default  1642.000000         hybrid  1642.000000  13.0000  13.0000
12    Default  1782.000000         indica  1782.000000  12.0000  12.0000
604   Default   168.000000         energy   168.000000  11.0000  11.0000
329   Default   339.000000          yield   339.000000  10.0000  10.0000
347   Default   323.000000           week   323.000000   9.0000   9.0000
152   Default   186.000000         orange   186.000000   8.0000   8.0000
499   Default   182.000000         skunky   182.000000   7.0000   7.0000
173   Default  1366.000000          cross  1366.000000   6.0000   6.0000
85    Default  1354.000000       dominant  1354.000000   5.0000   5.0000
482   Default   284.000000         grower   284.000000   4.0000   4.0000
145   Default  1340.000000           kush  1340.000000   3.0000   3.0000
132   Default  1303.000000          aroma  1303.000000   2.0000   2.0000
1033  Default   169.000000          cooky   169.000000   1.0000   1.0000
...       ...          ...            ...          ...      ...      ...
2300  Topic15     0.024732         medley     0.930781   2.4311  -9.1358
6827  Topic15     0.024720           pile     0.930469   2.4310  -9.1362
6825  Topic15     0.024720        hashier     0.930469   2.4310  -9.1362
6826  Topic15     0.024720  kushbreakfast     0.930469   2.4310  -9.1362
4170  Topic15     0.024720     exemplifie     0.930521   2.4309  -9.1362
1642  Topic15     0.024741     watermelon     0.931322   2.4309  -9.1354
1932  Topic15     0.024723     distribute     0.930706   2.4308  -9.1361
6270  Topic15     0.024695    seattlesoda     0.929710   2.4308  -9.1372
4738  Topic15     0.024723   vaporization     0.930881   2.4306  -9.1361
2964  Topic15     0.024748           seat     